 # Τεχνητή Νοημοσύνη: Εργαστηριακή Άσκηση 2 
---
## Γκέγκας Δημήτριος (03116004)
### 7ο Εξάμηνο
### dimitrisgegas01@gmail.com

---

# **Μέρος 1: Μελέτη των Metadata, Δημιουργία κόσμου και των βασικών queries.**


In [1]:
# Η βιβλιοθήκη pandas είναι χρήσιμη για την εργασία με τέτοια δεδομένα
import pandas as pd 
# Διάβασμα του αρχείου 'movie_metadata.csv' 
data = pd.read_csv("movie_metadata.csv") 
#Στο csv υπαρχούν κελία με nan τιμές
#Στις θέσεις αυτές βάζουμε 'UNK' πράγμα που  κάνουμε με την παρακάτω συνάρτηση
data.fillna("UNK", inplace=True)
# Preview the first 5 lines of the loaded data 
data.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723,178,0,855,Joel David Moore,1000,7.60506e+08,Action|Adventure|Fantasy|Sci-Fi,...,3054,English,USA,PG-13,2.37e+08,2009,936,7.9,1.78,33000
1,Color,Gore Verbinski,302,169,563,1000,Orlando Bloom,40000,3.09404e+08,Action|Adventure|Fantasy,...,1238,English,USA,PG-13,3e+08,2007,5000,7.1,2.35,0
2,Color,Sam Mendes,602,148,0,161,Rory Kinnear,11000,2.00074e+08,Action|Adventure|Thriller,...,994,English,UK,PG-13,2.45e+08,2015,393,6.8,2.35,85000
3,Color,Christopher Nolan,813,164,22000,23000,Christian Bale,27000,4.48131e+08,Action|Thriller,...,2701,English,USA,PG-13,2.5e+08,2012,23000,8.5,2.35,164000
4,UNK,Doug Walker,UNK,UNK,131,UNK,Rob Walker,131,UNK,Documentary,...,UNK,UNK,UNK,UNK,UNK,UNK,12,7.1,UNK,0


In [2]:
from pyswip import Prolog

In [3]:
def clean_text(text):
    text = text.replace(u'\xa0', u'')
    text = text.replace(u"'", u'')
    text = text.replace(u'"', u'')
    return text

## Προσθέτουμε γνωρίζματε στον κόσμο της prolog

In [4]:
#create World
#Ορίζουμε τον κόσμο μας
prolog = Prolog()
#και σβήνουμε ότι υπήρχε αποθηκευμένο από πρίν
prolog.query('make')

#Για κάθε row του πίνακα φτιάχνουμε τα κατηγοτήματα που θέλουμε να αποθηκέυσουμε
#αρχικά σε μια λίστα με το όνομα literals
literals = []
movie_score = {}
for row in data.itertuples(index=True, name='Pandas'):
    movie_title = clean_text(getattr(row, 'movie_title'))
    director = clean_text(getattr(row, 'director_name'))
    actor1 = clean_text(getattr(row, 'actor_1_name'))
    actor2 = clean_text(getattr(row, 'actor_2_name'))
    actor3 = clean_text(getattr(row, 'actor_3_name'))
    color = clean_text(getattr(row, 'color'))
    language = clean_text(getattr(row, 'language'))
    year = getattr(row, 'title_year')
    imdb_score = getattr(row, 'imdb_score')
    
    movie_score[movie_title] = 0
    
    if movie_title != "UNK":
        if director != "UNK":
            literals.append("director('"+ movie_title +"','" + director + "')")
        if actor1!= "UNK":
            literals.append("actor1('"+ movie_title +"','" + actor1 + "')")
        if actor2!= "UNK":
            literals.append("actor2('"+ movie_title +"','" + actor2 + "')")
        if actor3!= "UNK":
            literals.append("actor3('"+ movie_title +"','" + actor3 + "')")
        if color!= "UNK":
            literals.append("color('"+ movie_title +"','" + color + "')")
        if language!= "UNK":
            literals.append("language('"+ movie_title +"','" + language + "')")
        if year!= "UNK":
            literals.append("year('"+ movie_title +"'," + str(int(year)) + ")")
        if imdb_score!= "UNK":
            literals.append("imdb_score('"+ movie_title +"','" + str(imdb_score) + "')")
        for genre in getattr(row, 'genres').split("|"):
            literals.append("genre('"+ movie_title +"','"+ genre +"')")
        for keyword in getattr(row, 'plot_keywords').split("|"):
            literals.append("plot('"+ movie_title +"','"+ clean_text(keyword) +"')")

#Η Prolog θέλει τα κατηγορήματά της με την σειρά 
literals.sort()
for literal in literals:
    prolog.assertz(literal)

#Επίσης μπορούμε να κάνουμε consult ένα έτοιμό αρχίο στον κόσμο όπως γίνεται παρακάτω
# prolog.consult("filename.pl")

## Δημιουργούμε σύνθετους κανόνες prolog 

In [5]:
prolog.assertz('(actor(M,A):- actor1(M, A))')
prolog.assertz('(actor(M,A):- actor2(M, A))')
prolog.assertz('(actor(M,A):- actor3(M, A))')

prolog.assertz('(same_actor(X, Y,N) :-distinct(same_actor_distinct(X, Y,N)))')
prolog.assertz('(same_actor_distinct(X, Y,N) :-same_actor_range(X, Y,N),D is N+1,not(same_actor_range(X, Y,D)))')
prolog.assertz('(same_actor_range(X, Y,N) :-length(D,N),same_actor_range(X, Y,N,D),X\=Y)')
prolog.assertz('(same_actor_range(X, Y,N,[]):- N=:=0 )')
prolog.assertz('(same_actor_range(X, Y,N,[H|T]) :- actor(X,H), actor(Y,H) , same_actor_range(X,Y,N-1,T),not(member(H,T)) )  ')

In [6]:
prolog.assertz('(same_director(X, Y) :- director(X, D), director(Y, D),  X \= Y )')

In [7]:
prolog.assertz('(same_language(X, Y) :- language(X, D), language(Y, D),  X \= Y )')

In [8]:
prolog.assertz('(same_color(X, Y) :- color(X, D), color(Y, D),  X \= Y )')

In [9]:
prolog.assertz('(same_plot(X, Y,N) :-distinct(same_plot_distinct(X, Y,N)))')
prolog.assertz('(same_plot_distinct(X, Y,N) :-same_plot_range(X, Y,N),D is N+1,not(same_plot_range(X, Y,D)))')
prolog.assertz('(same_plot_range(X, Y,N) :-length(D,N),same_plot_range(X, Y,N,D),X\=Y)')
prolog.assertz('(same_plot_range(X, Y,N,[]):- N=:=0 )')
prolog.assertz('(same_plot_range(X, Y,N,[H|T]) :- plot(X,H), plot(Y,H) , same_plot_range(X,Y,N-1,T),not(member(H,T)) )  ')

In [10]:
prolog.assertz('(same_genre(X, Y,N) :-distinct(same_genre_distinct(X, Y,N)))')
prolog.assertz('(same_genre_distinct(X, Y,N) :-same_genre_range(X, Y,N),D is N+1,not(same_genre_range(X, Y,D)))')
prolog.assertz('(same_genre_range(X, Y,N) :-length(D,N),same_genre_range(X, Y,N,D),X\=Y)')
prolog.assertz('(same_genre_range(X, Y,N,[]):- N=:=0 )')
prolog.assertz('(same_genre_range(X, Y,N,[H|T]) :- genre(X,H), genre(Y,H) , same_genre_range(X,Y,N-1,T),not(member(H,T)) )  ')

In [11]:
prolog.assertz('(same_actor_range_d(X, Y,N) :-distinct(same_actor_range(X,Y,N)))')
prolog.assertz('(same_plot_range_d(X, Y,N) :-distinct(same_plot_range(X,Y,N)))')
prolog.assertz('(same_genre_range_d(X, Y,N) :-distinct(same_genre_range(X,Y,N)))')

### 1.	Αρκετά κοινό θέμα (κάποιες λέξεις σχετικά με το genre κοινές)

In [12]:
p=prolog.query("same_genre('Star Wars: Episode III - Revenge of the Sith',M,4)")
for s in p:
    print(s)

{'M': 'Aqua Teen Hunger Force Colon Movie Film for Theaters'}
{'M': 'Atlantis: The Lost Empire'}
{'M': 'Avatar'}
{'M': 'Beastmaster 2: Through the Portal of Time'}
{'M': 'Blade: Trinity'}
{'M': 'Destiny'}
{'M': 'Dragonball: Evolution'}
{'M': 'Final Fantasy: The Spirits Within'}
{'M': 'G-Force'}
{'M': 'Hellboy II: The Golden Army'}
{'M': 'Highlander: Endgame'}
{'M': 'Journey 2: The Mysterious Island'}
{'M': 'Journey to the Center of the Earth'}
{'M': 'Man of Steel'}
{'M': 'Megiddo: The Omega Code 2'}
{'M': 'Men in Black 3'}
{'M': 'Men in Black II'}
{'M': 'Mortal Kombat'}
{'M': 'Mortal Kombat: Annihilation'}
{'M': 'Pokémon 3: The Movie'}
{'M': 'Race to Witch Mountain'}
{'M': 'Reign of Fire'}
{'M': 'Star Wars: Episode I - The Phantom Menace'}
{'M': 'Star Wars: Episode II - Attack of the Clones'}
{'M': 'Star Wars: Episode IV - A New Hope'}
{'M': 'Star Wars: Episode V - The Empire Strikes Back'}
{'M': 'Star Wars: Episode VI - Return of the Jedi'}
{'M': 'Star Wars: The Clone Wars            

### 2.	Σχετικά κοινό θέμα (κάποιες λιγότερες λέξεις σχετικά με το genre κοινές π.χ. 3) 

In [13]:
p=prolog.query("same_genre('Star Wars: Episode III - Revenge of the Sith',M,3)")
for s in p:
    print(s)

{'M': '2012'}
{'M': '47 Ronin'}
{'M': '9'}
{'M': 'A Sound of Thunder'}
{'M': 'After Earth'}
{'M': 'Agent Cody Banks 2: Destination London'}
{'M': 'Aliens'}
{'M': 'Allegiant'}
{'M': 'Alpha and Omega 4: The Legend of the Saw Toothed Cave'}
{'M': 'Ant-Man'}
{'M': 'Armageddon'}
{'M': 'Avengers: Age of Ultron'}
{'M': 'Baahubali: The Beginning'}
{'M': 'Babylon A.D.'}
{'M': 'Batman Forever'}
{'M': 'Batman v Superman: Dawn of Justice'}
{'M': 'Battlefield Earth'}
{'M': 'Battleship'}
{'M': 'Beneath the Planet of the Apes'}
{'M': 'Beowulf'}
{'M': 'Big Hero 6'}
{'M': 'Big Trouble in Little China'}
{'M': 'BloodRayne'}
{'M': 'Captain America: Civil War'}
{'M': 'Captain America: The First Avenger'}
{'M': 'Captain America: The Winter Soldier'}
{'M': 'Cirque du Freak: The Vampires Assistant'}
{'M': 'Clash of the Titans'}
{'M': 'Clockstoppers'}
{'M': 'Cloverfield'}
{'M': 'Conan the Destroyer'}
{'M': 'Congo'}
{'M': 'Daredevil            '}
{'M': 'Dark City'}
{'M': 'Dawn of the Planet of the Apes'}
{'M': 

### 3.	Λιγότερο κοινό θέμα (λίγες λέξεις σχετικές με το genre κοινές π.χ. 1)

In [14]:
p=prolog.query("same_genre('Star Wars: Episode III - Revenge of the Sith',M,1)")
for s in p:
    print(s)

{'M': '12 Rounds'}
{'M': '13 Hours'}
{'M': '15 Minutes'}
{'M': '16 Blocks'}
{'M': '1941'}
{'M': '2 Fast 2 Furious'}
{'M': '2 Guns'}
{'M': '21 Jump Street'}
{'M': '22 Jump Street'}
{'M': '3 Days to Kill'}
{'M': '3 Ninjas Kick Back'}
{'M': '30 Minutes or Less'}
{'M': '3000 Miles to Graceland'}
{'M': '5 Days of War'}
{'M': 'A Fistful of Dollars'}
{'M': 'A Good Day to Die Hard'}
{'M': 'A Low Down Dirty Shame'}
{'M': 'A Man Apart'}
{'M': 'A Most Violent Year'}
{'M': 'Abduction'}
{'M': 'Absolute Power'}
{'M': 'Action Jackson'}
{'M': 'After the Sunset'}
{'M': 'Airlift'}
{'M': 'Alex Cross'}
{'M': 'All About the Benjamins'}
{'M': 'All the Queens Men'}
{'M': 'American Heist'}
{'M': 'American Ninja 2: The Confrontation'}
{'M': 'American Outlaws'}
{'M': 'American Sniper'}
{'M': 'Amidst the Devils Wings'}
{'M': 'Antitrust'}
{'M': 'Armored'}
{'M': 'Assassins'}
{'M': 'Assault on Precinct 13'}
{'M': 'Austin Powers in Goldmember'}
{'M': 'Bad Boys II'}
{'M': 'Bad Boys'}
{'M': 'Bandidas'}
{'M': 'Bangkok 

{'M': 'Queen of the Damned'}
{'M': 'Ratatouille'}
{'M': 'Ravenous'}
{'M': 'Red Riding Hood'}
{'M': 'Rubber'}
{'M': 'Sabrina, the Teenage Witch            '}
{'M': 'Saving Grace            '}
{'M': 'Scrooged'}
{'M': 'Seed of Chucky'}
{'M': 'Shallow Hal'}
{'M': 'Shorts'}
{'M': 'Simply Irresistible'}
{'M': 'Sliding Doors'}
{'M': 'Snow White and the Seven Dwarfs'}
{'M': 'Snow White: A Tale of Terror'}
{'M': 'Somewhere in Time'}
{'M': 'Son of the Mask'}
{'M': 'South Park: Bigger Longer & Uncut'}
{'M': 'Splash'}
{'M': 'Stranger Than Fiction'}
{'M': 'Teachers Pet'}
{'M': 'Ted'}
{'M': 'Teen Wolf Too'}
{'M': 'Teeth'}
{'M': 'The Addams Family'}
{'M': 'The Age of Adaline'}
{'M': 'The Blue Bird'}
{'M': 'The Box'}
{'M': 'The Brass Teapot'}
{'M': 'The Cabin in the Woods'}
{'M': 'The Call of Cthulhu'}
{'M': 'The Change-Up'}
{'M': 'The Cooler'}
{'M': 'The Craft'}
{'M': 'The Curious Case of Benjamin Button'}
{'M': 'The Evil Dead'}
{'M': 'The Family Man'}
{'M': 'The Fisher King'}
{'M': 'The Flintstones'

### 4. Κοινός σκηνοθέτης

In [15]:
p=prolog.query("same_director('Star Wars: Episode III - Revenge of the Sith',M)")
for s in p:
    print(s)

{'M': 'American Graffiti'}
{'M': 'Star Wars: Episode I - The Phantom Menace'}
{'M': 'Star Wars: Episode II - Attack of the Clones'}
{'M': 'Star Wars: Episode IV - A New Hope'}


### 5.	Ακριβώς ίδια πλοκή (κάποιες λέξεις κλειδιά της πλοκής να είναι κοινές) 

In [16]:
p=prolog.query("same_plot('Star Wars: Episode III - Revenge of the Sith',M,3)")
for s in p:
    print(s)

### 6.	Σχετικά ίδια πλοκή (κάποιες λιγότερες λέξεις κλειδιά κοινές)

In [17]:
p=prolog.query("same_plot('Star Wars: Episode III - Revenge of the Sith',M,1)")
for s in p:
    print(s)

{'M': 'Close Encounters of the Third Kind'}
{'M': 'The Dead Zone'}
{'M': 'The Twilight Saga: Breaking Dawn - Part 2'}


### 7.	Ίδιους τους βασικούς ηθοποιούς (και τους 3)

In [18]:
p=prolog.query("same_actor('Star Wars: Episode III - Revenge of the Sith',M,3)")
for s in p:
    print(s)

{'M': 'Star Wars: Episode II - Attack of the Clones'}


### 8.	Αρκετά ίδιους βασικούς ηθοποιούς (ορισμένους βασικούς ηθοποιούς κοινούς π.χ. 2) 

In [19]:
p=prolog.query("same_actor('Star Wars: Episode III - Revenge of the Sith',M,2)")
for s in p:
    print(s)

### 9.	Σχετικά ίδιους ηθοποιούς (π.χ. 1 από τους 3)

In [20]:
p=prolog.query("same_actor('Star Wars: Episode III - Revenge of the Sith',M,1)")
for s in p:
    print(s)

{'M': 'Anywhere But Here'}
{'M': 'Black Swan'}
{'M': 'Brothers'}
{'M': 'Closer'}
{'M': 'Jane Got a Gun'}
{'M': 'Mars Attacks!'}
{'M': 'My Blueberry Nights'}
{'M': 'No Strings Attached'}
{'M': 'Star Wars: Episode I - The Phantom Menace'}
{'M': 'The Other Boleyn Girl'}
{'M': 'V for Vendetta'}
{'M': 'Where the Heart Is'}
{'M': 'Your Highness'}
{'M': 'Cold Mountain'}
{'M': 'Hesher'}
{'M': 'Thor'}
{'M': 'Thor: The Dark World'}
{'M': '1941'}
{'M': 'Caravans'}
{'M': 'Gremlins 2: The New Batch'}
{'M': 'Police Academy: Mission to Moscow'}
{'M': 'The Golden Compass'}
{'M': 'The Lord of the Rings: The Fellowship of the Ring'}
{'M': 'The Lord of the Rings: The Two Towers'}
{'M': 'The Man with the Golden Gun'}
{'M': 'The Torture Chamber of Dr. Sadism'}
{'M': 'Charlie and the Chocolate Factory'}
{'M': 'Corpse Bride'}
{'M': 'Hugo'}
{'M': 'Dark Shadows'}
{'M': '90 Minutes in Heaven'}
{'M': 'American Heist'}
{'M': 'Awake'}
{'M': 'Factory Girl'}
{'M': 'Shattered Glass'}
{'M': 'Virgin Territory'}
{'M': '

###  10.	Ίδια γλώσσα 

In [21]:
p=prolog.query("same_language('Star Wars: Episode III - Revenge of the Sith',M)")
for s in p:
    print(s)

{'M': '#Horror'}
{'M': '10 Cloverfield Lane'}
{'M': '10 Days in a Madhouse'}
{'M': '10 Things I Hate About You'}
{'M': '102 Dalmatians'}
{'M': '10th & Wolf'}
{'M': '11:14'}
{'M': '12 Angry Men'}
{'M': '12 Monkeys            '}
{'M': '12 Rounds'}
{'M': '12 Years a Slave'}
{'M': '127 Hours'}
{'M': '13 Going on 30'}
{'M': '13 Hours'}
{'M': '1408'}
{'M': '15 Minutes'}
{'M': '16 Blocks'}
{'M': '16 to Life'}
{'M': '17 Again'}
{'M': '1776'}
{'M': '1941'}
{'M': '1982'}
{'M': '2 Fast 2 Furious'}
{'M': '2 Guns'}
{'M': '20 Dates'}
{'M': '20 Feet from Stardom'}
{'M': '20,000 Leagues Under the Sea'}
{'M': '20,000 Leagues Under the Sea'}
{'M': '200 Cigarettes'}
{'M': '2001: A Space Odyssey'}
{'M': '2012'}
{'M': '2016: Obamas America'}
{'M': '21 & Over'}
{'M': '21 Grams'}
{'M': '21 Jump Street'}
{'M': '21'}
{'M': '22 Jump Street'}
{'M': '24 7: Twenty Four Seven'}
{'M': '25th Hour'}
{'M': '27 Dresses'}
{'M': '28 Days Later...'}
{'M': '28 Days'}
{'M': '28 Weeks Later'}
{'M': '2:13'}
{'M': '3 Backyards'

{'M': 'Big Mommas House 2'}
{'M': 'Big Mommas House'}
{'M': 'Big Mommas: Like Father, Like Son'}
{'M': 'Big Trouble in Little China'}
{'M': 'Big Trouble'}
{'M': 'Big'}
{'M': 'Bill & Teds Bogus Journey'}
{'M': 'Bill & Teds Excellent Adventure'}
{'M': 'Billy Elliot'}
{'M': 'Birdman or (The Unexpected Virtue of Ignorance)'}
{'M': 'Birth'}
{'M': 'Birthday Girl'}
{'M': 'Bizarre'}
{'M': 'Black Christmas'}
{'M': 'Black Hawk Down'}
{'M': 'Black Knight'}
{'M': 'Black Mass'}
{'M': 'Black Nativity'}
{'M': 'Black November'}
{'M': 'Black Rain'}
{'M': 'Black Rock'}
{'M': 'Black Snake Moan'}
{'M': 'Black Swan'}
{'M': 'Black Water Transit'}
{'M': 'Black or White'}
{'M': 'Blackhat'}
{'M': 'Blackthorn'}
{'M': 'Blade II'}
{'M': 'Blade Runner'}
{'M': 'Blade'}
{'M': 'Blade: Trinity'}
{'M': 'Blades of Glory'}
{'M': 'Blast from the Past'}
{'M': 'Blazing Saddles'}
{'M': 'Bled'}
{'M': 'Bleeding Hearts'}
{'M': 'Blended'}
{'M': 'Bless the Child'}
{'M': 'Blindness'}
{'M': 'Blonde Ambition'}
{'M': 'Blood Diamond'}

{'M': 'Dark Water'}
{'M': 'Darkness Falls'}
{'M': 'Darkness'}
{'M': 'Darling Companion'}
{'M': 'Darling Lili'}
{'M': 'Date Movie'}
{'M': 'Date Night'}
{'M': 'Dave Chappelles Block Party'}
{'M': 'Dawn Patrol'}
{'M': 'Dawn of the Crescent Moon'}
{'M': 'Dawn of the Dead'}
{'M': 'Dawn of the Dead'}
{'M': 'Dawn of the Planet of the Apes'}
{'M': 'Day One'}
{'M': 'Day of the Dead'}
{'M': 'Day of the Dead'}
{'M': 'Daybreakers'}
{'M': 'Daylight'}
{'M': 'Days of Heaven'}
{'M': 'Days of Thunder'}
{'M': 'Dazed and Confused'}
{'M': 'De-Lovely'}
{'M': 'Dead Like Me: Life After Death'}
{'M': 'Dead Man Down'}
{'M': 'Dead Man Walking'}
{'M': 'Dead Man on Campus'}
{'M': 'Dead Mans Shoes'}
{'M': 'Dead Poets Society'}
{'M': 'Deadfall'}
{'M': 'Deadline - U.S.A.'}
{'M': 'Deadline Gallipoli            '}
{'M': 'Deadpool'}
{'M': 'Dear Frankie'}
{'M': 'Dear John'}
{'M': 'Dear Wendy'}
{'M': 'Death Becomes Her'}
{'M': 'Death Calls'}
{'M': 'Death Race 2000'}
{'M': 'Death Race'}
{'M': 'Death Sentence'}
{'M': 'Deat

{'M': 'Good'}
{'M': 'Goodfellas'}
{'M': 'Goosebumps'}
{'M': 'Goosebumps'}
{'M': 'Gory Gory Hallelujah'}
{'M': 'Gosford Park'}
{'M': 'Gossip'}
{'M': 'Gothika'}
{'M': 'Grabbers'}
{'M': 'Grace Unplugged'}
{'M': 'Grace of Monaco'}
{'M': 'Gracie'}
{'M': 'Graduation Day'}
{'M': 'Gran Torino'}
{'M': 'Grand Theft Parsons'}
{'M': 'Grandmas Boy'}
{'M': 'Grave Encounters'}
{'M': 'Gravity'}
{'M': 'Grease'}
{'M': 'Green Lantern'}
{'M': 'Green Room'}
{'M': 'Green Street 3: Never Back Down'}
{'M': 'Green Zone'}
{'M': 'Gremlins 2: The New Batch'}
{'M': 'Gremlins'}
{'M': 'Gridiron Gang'}
{'M': 'Griff the Invisible'}
{'M': 'Grindhouse'}
{'M': 'Groove'}
{'M': 'Grosse Pointe Blank'}
{'M': 'Groundhog Day'}
{'M': 'Growing Up Smith'}
{'M': 'Grown Ups 2'}
{'M': 'Grown Ups'}
{'M': 'Grudge Match'}
{'M': 'Guardians of the Galaxy'}
{'M': 'Guess Who'}
{'M': 'Guiana 1838'}
{'M': 'Gullivers Travels'}
{'M': 'Gun Shy'}
{'M': 'Gunless'}
{'M': 'H.'}
{'M': 'Hachi: A Dogs Tale'}
{'M': 'Hackers'}
{'M': 'Hail, Caesar!'}
{'M

{'M': 'Jaws: The Revenge'}
{'M': 'Jay and Silent Bob Strike Back'}
{'M': 'Jeepers Creepers II'}
{'M': 'Jeepers Creepers'}
{'M': 'Jeff, Who Lives at Home'}
{'M': 'Jefferson in Paris'}
{'M': 'Jekyll and Hyde... Together Again'}
{'M': 'Jennifers Body'}
{'M': 'Jerry Maguire'}
{'M': 'Jersey Boys'}
{'M': 'Jersey Girl'}
{'M': 'Jesse            '}
{'M': 'Jesus People'}
{'M': 'Jesus Son'}
{'M': 'Jimmy Neutron: Boy Genius'}
{'M': 'Jimmy and Judy'}
{'M': 'Jindabyne'}
{'M': 'Jingle All the Way'}
{'M': 'Joe Dirt'}
{'M': 'Joe Somebody'}
{'M': 'Joe'}
{'M': 'John Carter'}
{'M': 'John Q'}
{'M': 'Johnny English Reborn'}
{'M': 'Johnny English'}
{'M': 'Johnny Suede'}
{'M': 'Johnson Family Vacation'}
{'M': 'Jonah Hex'}
{'M': 'Jonah: A VeggieTales Movie'}
{'M': 'Josie and the Pussycats'}
{'M': 'Journey 2: The Mysterious Island'}
{'M': 'Journey to the Center of the Earth'}
{'M': 'Joy Ride'}
{'M': 'Joy'}
{'M': 'Joyful Noise'}
{'M': 'Judgment at Nuremberg'}
{'M': 'Julia'}
{'M': 'Julie & Julia'}
{'M': 'Jumper'}

...

{'M': 'Road House'}
{'M': 'Road Trip'}
{'M': 'Road to Perdition'}
{'M': 'Roadside'}
{'M': 'Roar'}
{'M': 'Rob Roy'}
{'M': 'Robin Hood'}
{'M': 'Robin Hood: Prince of Thieves'}
{'M': 'Robin and Marian'}
{'M': 'RoboCop 3'}
{'M': 'RoboCop'}
{'M': 'RoboCop'}
{'M': 'Robot & Frank'}
{'M': 'Robot Chicken            '}
{'M': 'Robots'}
{'M': 'Rock Star'}
{'M': 'Rock of Ages'}
{'M': 'Rockaway'}
{'M': 'RocknRolla'}
{'M': 'Rocky Balboa'}
{'M': 'Rocky'}
{'M': 'Rodeo Girl'}
{'M': 'Roger & Me'}
{'M': 'Rogue            '}
{'M': 'Role Models'}
{'M': 'Roll Bounce'}
{'M': 'Rollerball'}
{'M': 'Romance & Cigarettes'}
{'M': 'Romantic Schemer'}
{'M': 'Romeo + Juliet'}
{'M': 'Romeo Is Bleeding'}
{'M': 'Romeo Must Die'}
{'M': 'Ronin'}
{'M': 'Room'}
{'M': 'Rosemarys Baby'}
{'M': 'Rosewater'}
{'M': 'Rotor DR1'}
{'M': 'Rounders'}
{'M': 'Royal Kill'}
{'M': 'Rubber'}
{'M': 'Ruby in Paradise'}
{'M': 'Rudderless'}
{'M': 'Rugrats Go Wild'}
{'M': 'Rugrats in Paris: The Movie'}
{'M': 'Rules of Engagement            '}
{'M

{'M': 'The Assassination of Jesse James by the Coward Robert Ford'}
{'M': 'The Astronaut Farmer'}
{'M': 'The Astronauts Wife'}
{'M': 'The Astronauts Wife'}
{'M': 'The Avengers'}
{'M': 'The Avengers'}
{'M': 'The Aviator'}
{'M': 'The Awakening'}
{'M': 'The BFG'}
{'M': 'The Bachelor            '}
{'M': 'The Back-up Plan'}
{'M': 'The Bad News Bears'}
{'M': 'The Ballad of Cable Hogue'}
{'M': 'The Ballad of Gregorio Cortez'}
{'M': 'The Ballad of Jack and Rose'}
{'M': 'The Banger Sisters'}
{'M': 'The Bank Job'}
{'M': 'The Barbarians'}
{'M': 'The Basket'}
{'M': 'The Battle of Shaker Heights'}
{'M': 'The Beach'}
{'M': 'The Beast from 20,000 Fathoms'}
{'M': 'The Beastmaster'}
{'M': 'The Beaver'}
{'M': 'The Believer'}
{'M': 'The Benchwarmers'}
{'M': 'The Best Exotic Marigold Hotel'}
{'M': 'The Best Little Whorehouse in Texas'}
{'M': 'The Best Man Holiday'}
{'M': 'The Best Man'}
{'M': 'The Best Offer'}
{'M': 'The Best Years of Our Lives'}
{'M': 'The Best of Me'}
{'M': 'The Betrayed'}
{'M': 'The Bi

{'M': 'The Green Inferno'}
{'M': 'The Green Mile'}
{'M': 'The Grey'}
{'M': 'The Grudge 2'}
{'M': 'The Grudge'}
{'M': 'The Guard'}
{'M': 'The Guilt Trip'}
{'M': 'The Gunman'}
{'M': 'The Guru'}
{'M': 'The Hadza: Last of the First'}
{'M': 'The Hammer'}
{'M': 'The Hangover Part II'}
{'M': 'The Hangover'}
{'M': 'The Happening'}
{'M': 'The Harvest/La Cosecha'}
{'M': 'The Hateful Eight'}
{'M': 'The Haunted Mansion'}
{'M': 'The Haunting in Connecticut 2: Ghosts of Georgia'}
{'M': 'The Haunting of Molly Hartley'}
{'M': 'The Haunting'}
{'M': 'The Heart of Me'}
{'M': 'The Heat'}
{'M': 'The Hebrew Hammer'}
{'M': 'The Helix... Loaded'}
{'M': 'The Help'}
{'M': 'The Hills Have Eyes II'}
{'M': 'The Hills Have Eyes'}
{'M': 'The History Boys'}
{'M': 'The Hit List'}
{'M': 'The Hitchhikers Guide to the Galaxy'}
{'M': 'The Hoax'}
{'M': 'The Hobbit: An Unexpected Journey'}
{'M': 'The Hobbit: The Battle of the Five Armies'}
{'M': 'The Hobbit: The Desolation of Smaug'}
{'M': 'The Hole'}
{'M': 'The Holiday'}
{

{'M': 'The Past is a Grotesque Animal'}
{'M': 'The Patriot'}
{'M': 'The Peacemaker'}
{'M': 'The Peanuts Movie'}
{'M': 'The Pelican Brief'}
{'M': 'The Perez Family'}
{'M': 'The Perfect Game'}
{'M': 'The Perfect Host'}
{'M': 'The Perfect Man'}
{'M': 'The Perfect Match'}
{'M': 'The Perfect Storm'}
{'M': 'The Perfect Wave'}
{'M': 'The Perks of Being a Wallflower'}
{'M': 'The Pet'}
{'M': 'The Phantom of the Opera'}
{'M': 'The Phantom'}
{'M': 'The Pianist'}
{'M': 'The Piano'}
{'M': 'The Pink Panther'}
{'M': 'The Pirate'}
{'M': 'The Pirates Who Dont Do Anything: A VeggieTales Movie'}
{'M': 'The Pirates! Band of Misfits'}
{'M': 'The Place Beyond the Pines'}
{'M': 'The Player            '}
{'M': 'The Players Club'}
{'M': 'The Pledge'}
{'M': 'The Poker House'}
{'M': 'The Polar Express'}
{'M': 'The Possession'}
{'M': 'The Possession'}
{'M': 'The Postman Always Rings Twice'}
{'M': 'The Postman'}
{'M': 'The Powerpuff Girls            '}
{'M': 'The Prestige'}
{'M': 'The Prince and Me'}
{'M': 'The Pr

{'M': 'Unforgiven'}
{'M': 'Unforgotten            '}
{'M': 'Unfriended'}
{'M': 'United 93'}
{'M': 'United Passions'}
{'M': 'Universal Soldier: The Return'}
{'M': 'Unknown'}
{'M': 'Unknown'}
{'M': 'Unleashed'}
{'M': 'Unnatural'}
{'M': 'Unstoppable'}
{'M': 'Unsullied'}
{'M': 'Untraceable'}
{'M': 'Up Close & Personal'}
{'M': 'Up in the Air'}
{'M': 'Up'}
{'M': 'Urban Legend'}
{'M': 'Urban Legends: Final Cut'}
{'M': 'Urbania'}
{'M': 'V for Vendetta'}
{'M': 'Vacation'}
{'M': 'Valentine'}
{'M': 'Valentines Day'}
{'M': 'Valiant'}
{'M': 'Valkyrie'}
{'M': 'Valley of the Hearts Delight'}
{'M': 'Vampire Killers'}
{'M': 'Vampire in Brooklyn'}
{'M': 'Vampires Suck'}
{'M': 'Vampires'}
{'M': 'Vamps'}
{'M': 'Van Wilder: Party Liaison'}
{'M': 'Vanilla Sky'}
{'M': 'Vanity Fair'}
{'M': 'Vantage Point'}
{'M': 'Varsity Blues'}
{'M': 'Velvet Goldmine'}
{'M': 'Vera Drake'}
{'M': 'Veronica Guerin'}
{'M': 'Veronica Mars            '}
{'M': 'Veronika Decides to Die'}
{'M': 'Vertical Limit'}
{'M': 'Very Bad Thing

### 11.	Είναι έγχρωμες ή ασπρόμαυρες

In [22]:
p=prolog.query("same_color('Star Wars: Episode III - Revenge of the Sith',M)")
for s in p:
    print(s)

{'M': '#Horror'}
{'M': '10 Cloverfield Lane'}
{'M': '10 Days in a Madhouse'}
{'M': '10 Things I Hate About You'}
{'M': '102 Dalmatians'}
{'M': '10th & Wolf'}
{'M': '11:14'}
{'M': '12 Monkeys            '}
{'M': '12 Rounds'}
{'M': '12 Years a Slave'}
{'M': '127 Hours'}
{'M': '13 Going on 30'}
{'M': '13 Hours'}
{'M': '1408'}
{'M': '15 Minutes'}
{'M': '16 Blocks'}
{'M': '16 to Life'}
{'M': '17 Again'}
{'M': '1776'}
{'M': '1911'}
{'M': '1941'}
{'M': '1982'}
{'M': '2 Fast 2 Furious'}
{'M': '2 Guns'}
{'M': '20 Dates'}
{'M': '20 Feet from Stardom'}
{'M': '20,000 Leagues Under the Sea'}
{'M': '20,000 Leagues Under the Sea'}
{'M': '200 Cigarettes'}
{'M': '2001: A Space Odyssey'}
{'M': '2012'}
{'M': '2016: Obamas America'}
{'M': '21 & Over'}
{'M': '21 Grams'}
{'M': '21 Jump Street'}
{'M': '21'}
{'M': '22 Jump Street'}
{'M': '25th Hour'}
{'M': '27 Dresses'}
{'M': '28 Days Later...'}
{'M': '28 Days'}
{'M': '28 Weeks Later'}
{'M': '2:13'}
{'M': '3 Backyards'}
{'M': '3 Days to Kill'}
{'M': '3 Men an

{'M': 'Bad Santa'}
{'M': 'Bad Teacher'}
{'M': 'Bad Words'}
{'M': 'Baggage Claim'}
{'M': 'Baghead'}
{'M': 'Bait'}
{'M': 'Ballistic: Ecks vs. Sever'}
{'M': 'Bambi'}
{'M': 'Bamboozled'}
{'M': 'Bananas'}
{'M': 'Bandidas'}
{'M': 'Bandits'}
{'M': 'Bandslam'}
{'M': 'Bang Bang Baby'}
{'M': 'Bang'}
{'M': 'Bangkok Dangerous'}
{'M': 'Banshee Chapter'}
{'M': 'Barbarella'}
{'M': 'Barbecue'}
{'M': 'Barbershop'}
{'M': 'Barfi'}
{'M': 'Barneys Great Adventure'}
{'M': 'Barneys Version'}
{'M': 'Barnyard'}
{'M': 'Barry Lyndon'}
{'M': 'Barry Munday'}
{'M': 'Basic Instinct 2'}
{'M': 'Basic'}
{'M': 'Basquiat'}
{'M': 'Bathing Beauty'}
{'M': 'Bathory: Countess of Blood'}
{'M': 'Batman & Robin'}
{'M': 'Batman Begins'}
{'M': 'Batman Forever'}
{'M': 'Batman Returns'}
{'M': 'Batman v Superman: Dawn of Justice'}
{'M': 'Batman'}
{'M': 'Batman: The Dark Knight Returns, Part 2'}
{'M': 'Batman: The Movie'}
{'M': 'Bats'}
{'M': 'Battle Los Angeles'}
{'M': 'Battle for the Planet of the Apes'}
{'M': 'Battle of the Year'}
{

{'M': 'Closure'}
{'M': 'Cloud Atlas'}
{'M': 'Cloudy with a Chance of Meatballs 2'}
{'M': 'Cloudy with a Chance of Meatballs'}
{'M': 'Cloverfield'}
{'M': 'Club Dread'}
{'M': 'Clueless'}
{'M': 'Coach Carter'}
{'M': 'Coal Miners Daughter'}
{'M': 'Coco Before Chanel'}
{'M': 'Code 46'}
{'M': 'Code Name: The Cleaner'}
{'M': 'Code of Honor'}
{'M': 'Coffee Town'}
{'M': 'Cold Mountain'}
{'M': 'Collateral Damage'}
{'M': 'Collateral'}
{'M': 'College'}
{'M': 'Colombiana'}
{'M': 'Come Early Morning'}
{'M': 'Coming Home'}
{'M': 'Commando'}
{'M': 'Compadres'}
{'M': 'Compliance'}
{'M': 'Con Air'}
{'M': 'Conan the Barbarian'}
{'M': 'Conan the Barbarian'}
{'M': 'Conan the Destroyer'}
{'M': 'Concussion'}
{'M': 'Confessions of a Dangerous Mind'}
{'M': 'Confessions of a Teenage Drama Queen'}
{'M': 'Confidence'}
{'M': 'Congo'}
{'M': 'Connie and Carla'}
{'M': 'Conquest of the Planet of the Apes'}
{'M': 'Conspiracy Theory'}
{'M': 'Constantine            '}
{'M': 'Contact'}
{'M': 'Contagion'}
{'M': 'Containmen

{'M': 'Exotica'}
{'M': 'Extract'}
{'M': 'Extraordinary Measures'}
{'M': 'Extreme Measures'}
{'M': 'Extreme Movie'}
{'M': 'Extreme Ops'}
{'M': 'Extremely Loud & Incredibly Close'}
{'M': 'Eye See You'}
{'M': 'Eye for an Eye'}
{'M': 'Eye of the Beholder'}
{'M': 'Eye of the Dolphin'}
{'M': 'Eyes Wide Shut'}
{'M': 'F.I.S.T.'}
{'M': 'Face/Off'}
{'M': 'Facing the Giants'}
{'M': 'Factory Girl'}
{'M': 'Fahrenheit 9/11'}
{'M': 'Failure to Launch'}
{'M': 'Fair Game'}
{'M': 'Faith Connections'}
{'M': 'Faith Like Potatoes'}
{'M': 'Faithful'}
{'M': 'Falcon Rising'}
{'M': 'Fame'}
{'M': 'Family Plot'}
{'M': 'Fantasia 2000'}
{'M': 'Fantasia'}
{'M': 'Fantastic 4: Rise of the Silver Surfer'}
{'M': 'Fantastic Four'}
{'M': 'Fantastic Four'}
{'M': 'Fantastic Mr. Fox'}
{'M': 'Far from Heaven'}
{'M': 'Far from Men'}
{'M': 'Farce of the Penguins'}
{'M': 'Fargo            '}
{'M': 'Fascination'}
{'M': 'Fast Five'}
{'M': 'Fast Times at Ridgemont High'}
{'M': 'Faster'}
{'M': 'Fat Albert'}
{'M': 'Fat, Sick & Nearl

{'M': 'Hercules'}
{'M': 'Here Comes the Boom'}
{'M': 'Here on Earth'}
{'M': 'Hereafter'}
{'M': 'Heroes            '}
{'M': 'Heroes of Dirt'}
{'M': 'Hes Just Not That Into You'}
{'M': 'Hesher'}
{'M': 'Hey Arnold! The Movie'}
{'M': 'Hidalgo'}
{'M': 'Hidden Away'}
{'M': 'Hide and Seek'}
{'M': 'High Anxiety'}
{'M': 'High Crimes'}
{'M': 'High Fidelity'}
{'M': 'High Heels and Low Lifes'}
{'M': 'High Plains Drifter'}
{'M': 'High Road'}
{'M': 'High School Musical 2'}
{'M': 'High School Musical 3: Senior Year'}
{'M': 'High School Musical'}
{'M': 'High Tension'}
{'M': 'Higher Ground'}
{'M': 'Highlander'}
{'M': 'Highlander: Endgame'}
{'M': 'Highlander: The Final Dimension'}
{'M': 'Highway'}
{'M': 'History of the World: Part I'}
{'M': 'History of the World: Part I'}
{'M': 'Hit and Run'}
{'M': 'Hit the Floor            '}
{'M': 'Hitch'}
{'M': 'Hitman'}
{'M': 'Hits'}
{'M': 'Hobo with a Shotgun'}
{'M': 'Hocus Pocus'}
{'M': 'Hoffa'}
{'M': 'Hollow Man'}
{'M': 'Hollywood Ending'}
{'M': 'Hollywood Homici

...

{'M': 'Robin Hood: Prince of Thieves'}
{'M': 'Robin and Marian'}
{'M': 'RoboCop 3'}
{'M': 'RoboCop'}
{'M': 'RoboCop'}
{'M': 'Robot & Frank'}
{'M': 'Robot Chicken            '}
{'M': 'Robots'}
{'M': 'Rock Star'}
{'M': 'Rock of Ages'}
{'M': 'Rockaway'}
{'M': 'Rocket Singh: Salesman of the Year'}
{'M': 'RocknRolla'}
{'M': 'Rocky Balboa'}
{'M': 'Rocky'}
{'M': 'Rodeo Girl'}
{'M': 'Roger & Me'}
{'M': 'Rogue            '}
{'M': 'Role Models'}
{'M': 'Roll Bounce'}
{'M': 'Rollerball'}
{'M': 'Romance & Cigarettes'}
{'M': 'Romantic Schemer'}
{'M': 'Romeo + Juliet'}
{'M': 'Romeo Is Bleeding'}
{'M': 'Romeo Must Die'}
{'M': 'Ronin'}
{'M': 'Room'}
{'M': 'Rosemarys Baby'}
{'M': 'Rosewater'}
{'M': 'Rotor DR1'}
{'M': 'Rounders'}
{'M': 'Royal Kill'}
{'M': 'Rubber'}
{'M': 'Ruby in Paradise'}
{'M': 'Rudderless'}
{'M': 'Rugrats Go Wild'}
{'M': 'Rugrats in Paris: The Movie'}
{'M': 'Rules of Engagement            '}
{'M': 'Rumble in the Bronx'}
{'M': 'Run All Night'}
{'M': 'Run, Fatboy, Run'}
{'M': 'Run, Hide

{'M': 'Stealth'}
{'M': 'Steamboy'}
{'M': 'Steel'}
{'M': 'Step Brothers'}
{'M': 'Step Up 2: The Streets'}
{'M': 'Step Up 3D'}
{'M': 'Step Up Revolution'}
{'M': 'Step Up'}
{'M': 'Stepmom'}
{'M': 'Steppin: The Movie'}
{'M': 'Steve Jobs'}
{'M': 'Stick It'}
{'M': 'Stiff Upper Lips'}
{'M': 'Stigmata'}
{'M': 'Still Alice'}
{'M': 'Stir of Echoes'}
{'M': 'Stitches'}
{'M': 'Stoker'}
{'M': 'Stolen Summer'}
{'M': 'Stolen'}
{'M': 'Stomp the Yard'}
{'M': 'Stone Cold'}
{'M': 'Stone'}
{'M': 'Stonewall'}
{'M': 'Stop-Loss'}
{'M': 'Straight As'}
{'M': 'Straight Out of Brooklyn'}
{'M': 'Straight Outta Compton'}
{'M': 'Strange Wilderness'}
{'M': 'Stranger Than Fiction'}
{'M': 'Strangerland'}
{'M': 'Strangers with Candy            '}
{'M': 'Straw Dogs'}
{'M': 'Street Fighter'}
{'M': 'Street Fighter: The Legend of Chun-Li'}
{'M': 'Street Kings'}
{'M': 'Stripes'}
{'M': 'Striptease'}
{'M': 'Stuart Little 2'}
{'M': 'Stuart Little'}
{'M': 'Stuck on You'}
{'M': 'Stung'}
{'M': 'Subconscious'}
{'M': 'Sublime'}
{'M'

{'M': 'The Expendables 3'}
{'M': 'The Expendables'}
{'M': 'The Exploding Girl'}
{'M': 'The Extra Man'}
{'M': 'The Eye'}
{'M': 'The FP'}
{'M': 'The Face of an Angel'}
{'M': 'The Faculty'}
{'M': 'The Fall of the Roman Empire'}
{'M': 'The Family            '}
{'M': 'The Family Man'}
{'M': 'The Family Stone'}
{'M': 'The Fan'}
{'M': 'The Fast and the Furious'}
{'M': 'The Fast and the Furious'}
{'M': 'The Fast and the Furious'}
{'M': 'The Fast and the Furious: Tokyo Drift'}
{'M': 'The Fault in Our Stars'}
{'M': 'The Fifth Element'}
{'M': 'The Fifth Estate'}
{'M': 'The Fighter'}
{'M': 'The Final Destination'}
{'M': 'The Finest Hours'}
{'M': 'The Firm'}
{'M': 'The First Wives Club'}
{'M': 'The Fisher King'}
{'M': 'The Five-Year Engagement'}
{'M': 'The Flintstones in Viva Rock Vegas'}
{'M': 'The Flintstones'}
{'M': 'The Flower of Evil'}
{'M': 'The Flowers of War'}
{'M': 'The Fog'}
{'M': 'The Fog'}
{'M': 'The Following            '}
{'M': 'The Forbidden Kingdom'}
{'M': 'The Forest'}
{'M': 'The F

{'M': 'The Man'}
{'M': 'The Manchurian Candidate'}
{'M': 'The Marine 4: Moving Target'}
{'M': 'The Marine'}
{'M': 'The Martian'}
{'M': 'The Mask of Zorro'}
{'M': 'The Mask'}
{'M': 'The Masked Saint'}
{'M': 'The Master of Disguise'}
{'M': 'The Master'}
{'M': 'The Matador'}
{'M': 'The Matrix Reloaded'}
{'M': 'The Matrix Revolutions'}
{'M': 'The Matrix'}
{'M': 'The Maze Runner'}
{'M': 'The Mechanic'}
{'M': 'The Medallion'}
{'M': 'The Men Who Stare at Goats'}
{'M': 'The Merchant of Venice'}
{'M': 'The Messenger'}
{'M': 'The Messenger: The Story of Joan of Arc'}
{'M': 'The Messengers            '}
{'M': 'The Mexican'}
{'M': 'The Midnight Meat Train'}
{'M': 'The Mighty Ducks'}
{'M': 'The Mighty Macs'}
{'M': 'The Mighty'}
{'M': 'The Mirror Has Two Faces'}
{'M': 'The Missing            '}
{'M': 'The Missing Person'}
{'M': 'The Mist'}
{'M': 'The Molly Maguires'}
{'M': 'The Mongol King'}
{'M': 'The Monuments Men'}
{'M': 'The Mortal Instruments: City of Bones'}
{'M': 'The Motel'}
{'M': 'The Mothm

{'M': 'Transcendence'}
{'M': 'Transformers'}
{'M': 'Transformers: Age of Extinction'}
{'M': 'Transformers: Dark of the Moon'}
{'M': 'Transformers: Revenge of the Fallen'}
{'M': 'Transporter 2'}
{'M': 'Transsiberian'}
{'M': 'Trapeze'}
{'M': 'Trapped            '}
{'M': 'Trash'}
{'M': 'Travelers and Magicians'}
{'M': 'Treachery'}
{'M': 'Treading Water'}
{'M': 'Treasure Planet'}
{'M': 'Trees Lounge'}
{'M': 'Trekkies'}
{'M': 'Tremors'}
{'M': 'Triangle'}
{'M': 'Triple 9'}
{'M': 'Trippin'}
{'M': 'Tristram Shandy: A Cock and Bull Story'}
{'M': 'Trollhunter'}
{'M': 'Troop Beverly Hills'}
{'M': 'Tropic Thunder'}
{'M': 'Trouble with the Curve'}
{'M': 'Troy'}
{'M': 'Trucker'}
{'M': 'True Grit'}
{'M': 'True Lies'}
{'M': 'True Romance'}
{'M': 'Trust the Man'}
{'M': 'Trust'}
{'M': 'Truth or Die'}
{'M': 'Tsotsi'}
{'M': 'Tuck Everlasting'}
{'M': 'Tucker and Dale vs Evil'}
{'M': 'Tumbleweeds'}
{'M': 'Tupac: Resurrection'}
{'M': 'Turbo'}
{'M': 'Turbulence'}
{'M': 'Tusk'}
{'M': 'Twilight Zone: The Movie'

# **2ο Μέρος: Recommendation System με βάση μόνο τα χαρακτηριστικά των ταινιών.**

Στο σημείο αυτό με βάση τους κανόνες που κατασκευάστηκαν στο Μέρος 1 θα κατασκευαστούν κατηγορήματα για την ομοιότητα ταινιών. Παρακάτω δίνεται ένα μικρό παράδειγμα ενός κανόνα και πώς αυτός θα μπορούσε να γραφτεί μέσω του Pyswip. Επίσης όπως αναφέρεται και σε σχόλιο παραπάνω θα μπορεί να γραφτεί και μια βάση δεδομένων με τους κανόνες και να γίνει απευθείας consult.

Στο παρακάτω παράδειγμα το 5 και το 4 εκφράζουν την ομοιότητα των ταινιών π.χ. οι ταινίες που παράγονται μέσω του find_similar_5 είναι πιο όμοιες από αυτές που παράγονται μέσω του find_similar_4. 


In [23]:
prolog.consult('db.pl')

In [25]:
def simple_recommender(movie):
    ans = []
    for i in range(5,0,-1):
        q = prolog.query("find_similar_movies('" + movie +"',M,"+str(i)+")")
        for soln in q:
            if soln["M"] not in ans:
                ans.append(soln["M"])
        q.close()

    return ans[:10]

## Στο σημείο αυτό θα ζητήσουμε από το σύστημα να μας επιστρέψει ορισμένα αποτελέσματα για ταινίες για να εξετάσουμε την απόδοσή του.


In [26]:
simple_recommender("Star Wars: Episode III - Revenge of the Sith")

['Star Wars: Episode I - The Phantom Menace',
 'Star Wars: Episode II - Attack of the Clones',
 'Star Wars: Episode IV - A New Hope',
 'The Twilight Saga: Breaking Dawn - Part 2',
 'Mars Attacks!',
 'Your Highness',
 'Thor',
 'Thor: The Dark World',
 'Caravans',
 'The Golden Compass']

In [27]:
simple_recommender("Avatar")

['Aliens',
 'Terminator 2: Judgment Day',
 'The Abyss',
 'The Terminator',
 'True Lies',
 'The Last Airbender',
 'Automata',
 'Chappie',
 'Enders Game',
 'In Time']

In [28]:
simple_recommender('Pirates of the Caribbean: Dead Mans Chest')

['Pirates of the Caribbean: At Worlds End',
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 'Rango',
 'The Lone Ranger',
 'The Mexican',
 'Gullivers Travels',
 'Rise of the Guardians',
 'Star Trek',
 'The Smurfs 2',
 'A Monster in Paris']

In [49]:
simple_recommender('Saw II')

['Alleluia! The Devils Carnival',
 'Repo! The Genetic Opera',
 'Saw III',
 'Saw IV',
 'Case 39',
 'Nine Dead',
 'One Missed Call',
 'Red Riding Hood',
 'After.Life',
 'Cry_Wolf']

In [29]:
def jaccard_similarity(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

### Παρακάτω βλέπουμε το πόσο σχετικά είναι τα αποτελέσματα του δικού μας recommender σε σχέδη με τα δοσμένα data.

In [30]:
relative = pd.read_csv("relative_movies.csv") 
sum =0
count=0
for row in relative.itertuples(index=True, name='Pandas'):
    relative_movies=[]
    count+=1
    movie = clean_text(getattr(row, 'movie_title'))
    for relative_movie in getattr(row, 'relative_movies').split(","):
            relative_movies.append(relative_movie )
    sum+= jaccard_similarity(simple_recommender(movie), relative_movies)
print(sum/count)


0.05282295356962994


##### Έγινε όσο το δινατό καλύτερη βελτίωση σε αυτό το νούμερο. Καθώς ξεκινήσαμε με 0.01

# **3ο Μέρος: Recommendation System Με βάση τις προτιμήσεις-αξιολογήσεις του χρήστη**

Αρχικά μελετάμε τις αξιολογήσεις κάθε χρήστη για να καταλάβουμε την δομή και τις πληροφορίες κάθε αρχείου.






In [31]:
similarity_weights = {5: 25, 4:12,3:5,2:2,1:1}
rating_weight = {5: 10, 4: 5, 3: 0, 2: -5, 1: -10}

**Στο σημείο αυτό κατασκευάζουμε την νέα συνάρτηση η όποια θα επιστρέφει τις προτεινόμενες ταινίες συνυπολογίζοντας τις αξιολογήσεις του χρήστη.**

In [32]:
def recommender(user,movie_score): 
    ratings = pd.read_csv(user) 
    movie_score = {m: 0 for m in  movie_score}
    for row in ratings.itertuples(index=True, name='Pandas'):
        movie = clean_text(getattr(row, 'movie_title'))
        rating = getattr(row, 'rating')    
        
        for i in range(5,0,-1):
            q = prolog.query("find_similar_movies('" + movie +"',M,"+str(i)+")")
            for soln in q:
                movie_score [soln["M"]] += similarity_weights[i] * rating_weight[rating] #το weight θα το ορίσετε ανα επίπεδο ομοιότητας οι πολύ όμοιες ταινίες θα έχουν μεγαλύτερο βάρος
            q.close()
    movies_score = {k : v for k, v in sorted(movie_score.items(), key=lambda item: item[1], reverse = True)}
    return [m for m in movies_score][:10]


In [33]:
movie_score = {m: 0 for m in  movie_score}
recommender("user1.csv",movie_score)

['John Carter',
 'Monsters vs. Aliens',
 'The Mummy Returns',
 'Green Lantern',
 'Planet 51',
 'War of the Worlds',
 'Pirates of the Caribbean: At Worlds End',
 'Rango',
 'Space Chimps',
 'Pirates of the Caribbean: The Curse of the Black Pearl']

In [34]:
movie_score = {m: 0 for m in  movie_score}
recommender("user2.csv",movie_score)

['My Soul to Take',
 '88 Minutes',
 'The Gunman',
 'Death Sentence',
 'Money Monster',
 'Blood Ties',
 'Harry Brown',
 'Rampage',
 'Eagle Eye',
 'Street Kings']

## Ο user1 έχει προτιμήσεις σε ταινίες φαντασίας και δράσεις. Ενώ ο user2 προτιμά thrillers και μυστήριο. Οι προτεινόμενες ταινίες σχετίζονται αρκετά καλά μεταξύ τους 

---
<center><h2>Κατασκευή των Δικών σας Users</h2></center>

---

In [38]:
import csv
import numpy as np

def create_my_user(liked, notLiked, filename, num_of_liked =20, num_of_notliked = 20):
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["movie_title","rating"])
        likedd = np.random.choice(liked, num_of_liked)
        notlikedd = np.random.choice(notliked, num_of_notliked)
        for i in  likedd:
            grade = np.random.choice([5,4,3], p = [0.5, 0.4, 0.1])
            writer.writerow([i, grade])
        for i in  notlikedd:
            grade = np.random.choice([1, 2, 3], p = [0.5, 0.4, 0.1])
            writer.writerow([i, grade])

# User με προτίμηση σε ταινίες τρόμο, δεν του αρέσουν η φαντασίας

In [39]:
prolog.assertz("(fantasy(X) :- genre(X, 'Sci-Fi'); genre(X, 'Fantacy');genre(X, 'Action'))")
prolog.assertz("(thriller(X) :- genre(X, 'Horror'),genre(X, 'Thriller'))")
prolog.assertz("(liked1(X) :- thriller(X))")
prolog.assertz("(notLiked1(X) :- fantasy(X))")
q = prolog.query("liked1(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked = list(set(ans))

q = prolog.query("notLiked1(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked = list(set(ans))

create_my_user(liked, notliked, filename= "myuserThriller.csv", num_of_liked =40, num_of_notliked = 40)
movie_score = {m: 0 for m in  movie_score}

In [40]:
movie_score = {m: 0 for m in  movie_score}
recommender("myuserThriller.csv",movie_score)

['Insidious',
 'The Conjuring 2',
 'eXistenZ',
 'The Dead Zone',
 'Shark Night 3D',
 'Spider',
 'The Conjuring',
 'Eden Lake',
 'A Dangerous Method',
 'A History of Violence']

# User με προτίμηση σε ταινίες φαντασίας και ταινίες που έχει σκηνοθετήσει ο James Cameron. Δεν του άρεσαν ταινίες παρόμιες με Pirates of the Caribbean

In [41]:
prolog.assertz("(liked2(X) :- fantasy(X);director(X,'James Cameron'))")
prolog.assertz("(notLiked2(X) :- find_similar_movies('Pirates of the Caribbean: Dead Mans Chest',X,5);find_similar_movies('Pirates of the Caribbean: Dead Mans Chest',X,4) )")
q = prolog.query("liked2(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked = list(set(ans))

q = prolog.query("notLiked2(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked = list(set(ans))

create_my_user(liked, notliked, filename= "myuserFantasy.csv", num_of_liked =40, num_of_notliked = 40)
movie_score = {m: 0 for m in  movie_score}

In [42]:
movie_score = {m: 0 for m in  movie_score}
recommender("myuserFantasy.csv",movie_score)

['Insomnia',
 'Inception',
 'Batman Begins',
 'The Dark Knight',
 'The Prestige',
 'Memento',
 'Batman v Superman: Dawn of Justice',
 'Brick Mansions',
 'A Good Day to Die Hard',
 'Stolen']

# User με προτίμηση σε ταινίες που έχουν σχέση με alien. Δεν του άρεσαν ταινίες τρόμου.

In [43]:
prolog.assertz("(liked3(X) :- plot(X,'alien'))")
prolog.assertz("(notLiked3(X) :- thriller(X))")
q = prolog.query("liked3(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
liked = list(set(ans))

q = prolog.query("notLiked3(M)")
ans = []
for soln in q:
    ans.append(soln["M"])
notliked = list(set(ans))

create_my_user(liked, notliked, filename= "myuserSpace.csv", num_of_liked =40, num_of_notliked = 40)
movie_score = {m: 0 for m in  movie_score}

In [44]:
movie_score = {m: 0 for m in  movie_score}
recommender("myuserSpace.csv",movie_score)

['The 5th Wave',
 'The Host',
 'The Darkest Hour',
 'Men in Black 3',
 'Attack the Block',
 'Race to Witch Mountain',
 'Edge of Tomorrow',
 'Pixels',
 'Escape from Planet Earth',
 'Independence Day: Resurgence']